In [44]:
!brew install poppler

⠋ JSON API formula.jws.json                        [Downloading  32.1MB/-------]
⠋ JSON API cask.jws.json                           [Downloading  14.8MB/-------]⠋ JSON API formula.jws.json                        [Downloading  32.1MB/-------]
⠋ JSON API cask.jws.json                           [Downloading  14.8MB/-------]⠙ JSON API formula.jws.json                        [Downloading  32.1MB/-------]
⠙ JSON API cask.jws.json                           [Downloading  14.8MB/-------]⠙ JSON API formula.jws.json                        [Downloading  32.1MB/-------]
⠙ JSON API cask.jws.json                           [Downloading  14.8MB/-------]⠚ JSON API formula.jws.json                        [Downloading   2.2MB/-------]
⠚ JSON API cask.jws.json                           [Downloading   2.2MB/-------]⠚ JSON API formula.jws.json                        [Downloading   9.2MB/-------]
⠚ JSON API cask.jws.json                           [Downloading   7.6MB/-------]⠞ JSON API formula.jws.json       

In [45]:
!pip install -q requests beautifulsoup4 python-dotenv google-genai pdf2image ddtrace openinference-instrumentation-google-genai

In [46]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote
from pdf2image import convert_from_path

def get_pdf_links(target_url):
    """ ดึงลิงก์ PDF ทั้งหมดจากหน้าเว็บ """
    pdf_links = []
    try:
        response = requests.get(target_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.lower().strip().endswith('.pdf'):
                full_url = urljoin(target_url, href)
                pdf_links.append(full_url)
        return pdf_links
    except Exception as e:
        print(f"Error fetching links: {e}")
        return []

def download_pdfs(pdf_links, pdf_folder):
    """ ดาวน์โหลด PDF ลงโฟลเดอร์ """
    if not os.path.exists(pdf_folder):
        os.makedirs(pdf_folder)
    
    print(f"Starting download of {len(pdf_links)} files...")
    
    downloaded_files = []
    for i, link in enumerate(pdf_links, 1):
        try:
            filename = unquote(os.path.basename(link))
            file_path = os.path.join(pdf_folder, filename)
            
            # เช็คว่ามีไฟล์อยู่แล้วหรือยัง (จะได้ไม่โหลดซ้ำ)
            if not os.path.exists(file_path):
                print(f"[{i}/{len(pdf_links)}] Downloading: {filename} ...", end=" ")
                response = requests.get(link, stream=True)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print("Done.")
            else:
                print(f"[{i}/{len(pdf_links)}] Skipping (Exists): {filename}")
                
            downloaded_files.append(file_path)
            
        except Exception as e:
            print(f"Failed to download {link}. Error: {e}")
            
    return downloaded_files

def convert_pdfs_to_images(pdf_files, image_folder, dpi=300):
    """ 
    แปลงไฟล์ PDF เป็นรูปภาพ (JPG) 
    Args:
        pdf_files (list): รายชื่อ path ของไฟล์ PDF
        image_folder (str): โฟลเดอร์ปลายทางที่จะเก็บรูป
        dpi (int): ความละเอียดของรูป (300 ชัดเจนดีสำหรับ OCR/Vision AI)
    """
    if not os.path.exists(image_folder):
        os.makedirs(image_folder)
        
    print(f"\nConverting {len(pdf_files)} PDFs to images...")
    
    for i, pdf_path in enumerate(pdf_files, 1):
        try:
            filename = os.path.basename(pdf_path)
            base_name = os.path.splitext(filename)[0]
            print(f"[{i}/{len(pdf_files)}] Converting: {filename} ...")
            
            # แปลง PDF เป็น list ของ images (1 หน้า = 1 รูป)
            # หมายเหตุ: ถ้า Windows หา poppler ไม่เจอ อาจต้องใส่ poppler_path=r'C:\path\to\poppler\bin'
            pages = convert_from_path(pdf_path, dpi=dpi)
            
            for page_num, page_image in enumerate(pages, 1):
                # ตั้งชื่อไฟล์รูป: ชื่อเดิม_page1.jpg
                image_name = f"{base_name}_page{page_num}.jpg"
                image_path = os.path.join(image_folder, image_name)
                
                # Save รูปภาพ
                page_image.save(image_path, 'JPEG')
                
        except Exception as e:
            print(f"Error converting {filename}: {e}")

def convert_local_pdfs_to_images(source_dir, destination_dir, dpi=350):
    """
    Reads all PDF files from a source directory and converts them to images.
    
    Args:
        source_dir (str): Path to the folder containing PDF files.
        destination_dir (str): Path to the folder where images will be saved.
        dpi (int): Resolution of the output images (300 is good for AI/OCR).
    """
    # 1. Create destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        print(f"Created output folder: {destination_dir}")

    # 2. Get list of all PDF files in the source directory
    # Using list comprehension to filter only .pdf files
    pdf_files = [f for f in os.listdir(source_dir) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"No PDF files found in '{source_dir}'")
        return

    print(f"Found {len(pdf_files)} PDF files in '{source_dir}'. Starting conversion...\n")

    # 3. Loop through each PDF and convert
    for i, filename in enumerate(pdf_files, 1):
        pdf_path = os.path.join(source_dir, filename)
        base_name = os.path.splitext(filename)[0]  # Remove .pdf extension
        
        try:
            print(f"[{i}/{len(pdf_files)}] Converting: {filename} ...", end=" ")
            
            # Convert PDF to list of images (one per page)
            # Note: For Windows, you might need: poppler_path=r'C:\path\to\poppler\bin'
            pages = convert_from_path(pdf_path, dpi=dpi)
            
            # Save each page as an image
            for page_num, page_image in enumerate(pages, 1):
                image_name = f"{base_name}_page{page_num}.jpg"
                save_path = os.path.join(destination_dir, image_name)
                
                page_image.save(save_path, 'JPEG')
            
            print(f"Done ({len(pages)} pages)")
            
        except Exception as e:
            print(f"\nError converting {filename}: {e}")

In [47]:
# URL to scrape
target_url = "https://webportal.bangkok.go.th/bangphlat/page/sub/26952/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%9C%E0%B8%A5%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%99%E0%B8%B1%E0%B8%9A%E0%B8%84%E0%B8%B0%E0%B9%81%E0%B8%99%E0%B8%99%E0%B8%AA%E0%B8%A1%E0%B8%B2%E0%B8%8A%E0%B8%B4%E0%B8%81%E0%B8%AA%E0%B8%A0%E0%B8%B2%E0%B8%9C%E0%B8%B9%E0%B9%89%E0%B9%81%E0%B8%97%E0%B8%99%E0%B8%A3%E0%B8%B2%E0%B8%A9%E0%B8%8E%E0%B8%A3"

# Name of the folder to save files
output_dir = "assets/ss5-18/ss5-18-pdf"

print("Step 1: Fetching links...")
links = get_pdf_links(target_url)

if links:
    print(f"Found {len(links)} PDFs. Proceeding to download.")
    print("-" * 40)
    # Call the new download function
    download_pdfs(links, output_dir)
    print("-" * 40)
    print(f"All downloads complete. Check the folder: '{output_dir}'")
else:
    print("No PDF links found.")

Step 1: Fetching links...
Found 118 PDFs. Proceeding to download.
----------------------------------------
Starting download of 118 files...
[1/118] Skipping (Exists): คู่มือผู้ใช้งานระบบหลังบ้าน_Bangkok-PORTAL.pdf
[2/118] Skipping (Exists): บางอ้อ1.pdf
[3/118] Skipping (Exists): บางอ้อ17.pdf
[4/118] Skipping (Exists): บางพลัด1.pdf
[5/118] Skipping (Exists): บางพลัด17.pdf
[6/118] Skipping (Exists): บางบำหรุ1.pdf
[7/118] Skipping (Exists): บางบำหรุ17.pdf
[8/118] Skipping (Exists): บางยี่ขัน1.pdf
[9/118] Skipping (Exists): บางยี่ขัน17.pdf
[10/118] Skipping (Exists): บางอ้อ2.pdf
[11/118] Skipping (Exists): บางอ้อ18.pdf
[12/118] Skipping (Exists): บางพลัด2.pdf
[13/118] Skipping (Exists): บางพลัด18.pdf
[14/118] Skipping (Exists): บางบำหรุ2.pdf
[15/118] Skipping (Exists): บางบำหรุ18.pdf
[16/118] Skipping (Exists): บางยี่ขัน2.pdf
[17/118] Skipping (Exists): บางยี่ขัน18.pdf
[18/118] Skipping (Exists): บางอ้อ3.pdf
[19/118] Skipping (Exists): บางอ้อ19.pdf
[20/118] Skipping (Exists): บางพลัด3.pdf

In [48]:
# --- Configuration ---
# Change these paths to match your actual folders
input_pdf_folder = "assets/ss5-18/ss5-18-pdf"       # Source folder with PDFs
output_image_folder = "assets/ss5-18/ss5-18-images"    # Destination for images

# Check if source directory exists before running
if os.path.exists(input_pdf_folder):
    convert_local_pdfs_to_images(input_pdf_folder, output_image_folder)
    
    print("-" * 40)
    print(f"All operations complete. Check folder: '{output_image_folder}'")
else:
    print(f"Error: Source directory '{input_pdf_folder}' not found.")

Found 118 PDF files in 'assets/ss5-18/ss5-18-pdf'. Starting conversion...

[1/118] Converting: บางยี่ขัน1.pdf ... Done (6 pages)
[2/118] Converting: บางยี่ขัน27.pdf ... Done (6 pages)
[3/118] Converting: บางยี่ขัน26.pdf ... Done (6 pages)
[4/118] Converting: บางบำหรุ1.pdf ... Done (6 pages)
[5/118] Converting: บางบำหรุ3.pdf ... Done (6 pages)
[6/118] Converting: บางยี่ขัน18.pdf ... Done (6 pages)
[7/118] Converting: บางยี่ขัน2.pdf ... Done (6 pages)
[8/118] Converting: บางยี่ขัน24.pdf ... Done (6 pages)
[9/118] Converting: บางยี่ขัน30.pdf ... Done (6 pages)
[10/118] Converting: บางยี่ขัน31.pdf ... Done (6 pages)
[11/118] Converting: บางยี่ขัน25.pdf ... Done (6 pages)
[12/118] Converting: บางยี่ขัน3.pdf ... Done (6 pages)
[13/118] Converting: บางยี่ขัน19.pdf ... Done (6 pages)
[14/118] Converting: บางบำหรุ2.pdf ... Done (6 pages)
[15/118] Converting: บางบำหรุ6.pdf ... Done (6 pages)
[16/118] Converting: บางยี่ขัน21.pdf ... Done (6 pages)
[17/118] Converting: บางยี่ขัน7.pdf ... Done (6 p

KeyboardInterrupt: 

# Gemini Multi-modal

In [1]:
!pip install -q python-dotenv google-genai ddtrace openinference-instrumentation-google-genai

In [18]:
!export DD_TRACE_AGENT_URL=http://localhost:8136


thread 'tokio-runtime-worker' (20225108) panicked at /Users/runner/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/tokio-1.48.0/src/runtime/io/driver.rs:196:23:
unexpected error when polling the I/O driver: Os { code: 9, kind: Uncategorized, message: "Bad file descriptor" }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace

thread 'tokio-runtime-worker' (20225109) panicked at /Users/runner/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/tokio-1.48.0/src/runtime/io/driver.rs:260:27:
failed to wake I/O driver: Os { code: 9, kind: Uncategorized, message: "Bad file descriptor" }


In [27]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env", override=True)


True

In [28]:
print("GOOGLE_CLOUD_PROJECT:", os.getenv("GOOGLE_CLOUD_PROJECT"))
print("GOOGLE_CLOUD_LOCATION:", os.getenv("GOOGLE_CLOUD_LOCATION"))
print("GOOGLE_GENAI_USE_VERTEXAI:", os.getenv("GOOGLE_GENAI_USE_VERTEXAI"))
print("DD_LLMOBS_ENABLED:", os.getenv("DD_LLMOBS_ENABLED"))
print("DD_LLMOBS_ML_APP:", os.getenv("DD_LLMOBS_ML_APP"))
print("DD_SITE:", os.getenv("DD_SITE"))
print("DD_SERVICE:", os.getenv("DD_SERVICE"))
print("DD_ENV:", os.getenv("DD_ENV"))
print("DD_VERSION:", os.getenv("DD_VERSION"))
print("DD_TRACE_AGENT_URL:", os.getenv("DD_TRACE_AGENT_URL"))
print("DD_TRACE_AGENT_PORT:", os.getenv("DD_TRACE_AGENT_PORT"))

print("OTEL_EXPORTER_OTLP_TRACES_PROTOCOL:", os.getenv("OTEL_EXPORTER_OTLP_TRACES_PROTOCOL"))
print("OTEL_EXPORTER_OTLP_TRACES_ENDPOINT:", os.getenv("OTEL_EXPORTER_OTLP_TRACES_ENDPOINT"))
print("OTEL_EXPORTER_OTLP_TRACES_HEADERS:", os.getenv("OTEL_EXPORTER_OTLP_TRACES_HEADERS"))
print("OTEL_SERVICE_NAME:", os.getenv("OTEL_SERVICE_NAME"))



GOOGLE_CLOUD_PROJECT: datadog-ese-sandbox
GOOGLE_CLOUD_LOCATION: global
GOOGLE_GENAI_USE_VERTEXAI: True
DD_LLMOBS_ENABLED: 1
DD_LLMOBS_ML_APP: vote-extractor
DD_SITE: datadoghq.com
DD_SERVICE: vote-extractor
DD_ENV: dev
DD_VERSION: 0.0.1
DD_TRACE_AGENT_URL: http://localhost:8136
DD_TRACE_AGENT_PORT: 8136
OTEL_EXPORTER_OTLP_TRACES_PROTOCOL: None
OTEL_EXPORTER_OTLP_TRACES_ENDPOINT: None
OTEL_EXPORTER_OTLP_TRACES_HEADERS: None
OTEL_SERVICE_NAME: None


In [26]:
from ddtrace.llmobs import LLMObs
LLMObs.enable(
  ml_app=os.getenv("DD_LLMOBS_ML_APP"),
  api_key=os.getenv("DD_API_KEY"),
  service=os.getenv("DD_SERVICE"),
  agentless_enabled=True,
)

In [29]:
import os
import re
import json
from google import genai
from google.genai import types

# --- 1. Helper Function: Group and Sort Images ---
def get_district_image_groups(directory, district_name):
    """
    Scans a directory for image files matching a specific district name,
    groups them by their report ID, and sorts them by page number.

    Args:
        directory (str): The path to the directory containing image files.
        district_name (str): The name of the district to filter by (e.g., "Bang Phlat").

    Returns:
        dict: A dictionary where keys are report IDs (e.g., "BangPhlat25")
              and values are lists of sorted file paths.
              Example: { "BangPhlat25": ["path/to/p1.jpg", "path/to/p2.jpg"] }
    """
    grouped_images = {}
    
    if not os.path.exists(directory):
        print(f"Error: Directory '{directory}' not found.")
        return {}

    # List all files in the directory
    files = os.listdir(directory)
    
    for filename in files:
        # Filter for image files and the specific district name
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')) and district_name in filename:
            
            # Logic to extract Report ID:
            # Assuming filename format like "District25_page1.jpg",
            # we split by "_page" to get "District25" as the group key.
            if "_page" in filename:
                report_id = filename.split("_page")[0]
                
                if report_id not in grouped_images:
                    grouped_images[report_id] = []
                
                full_path = os.path.join(directory, filename)
                grouped_images[report_id].append(full_path)

    # Sort images within each group based on the page number in the filename
    for report_id in grouped_images:
        # Use regex to find '_pageX' and extract the integer X for sorting
        grouped_images[report_id].sort(
            key=lambda x: int(re.search(r'_page(\d+)', x).group(1)) if re.search(r'_page(\d+)', x) else 0
        )
        
    return grouped_images

# --- 2. Gemini Function: Multi-page Extraction ---
def extract_from_multipage_report(image_paths, client):
    """
    Sends multiple document pages to Gemini for extraction, following best practices:
    1. Indexes each image (e.g., "Page 1").
    2. Places all images BEFORE the main text prompt.

    Args:
        image_paths (list): A list of file paths to the images (sorted).
        client (genai.Client): The initialized Google Gen AI client.

    Returns:
        dict: The extracted data as a JSON object, or None if extraction fails.
    """
    
    # List to hold all content parts (Text labels + Image bytes)
    content_parts = []
    
    # A. Process Images (Loop through sorted paths)
    for i, path in enumerate(image_paths, 1):
        try:
            filename = os.path.basename(path)
            
            # 1. Best Practice: Add an Index Label BEFORE the image.
            # This helps the model identify which page contains specific data (e.g., header vs table).
            index_label = f"Page {i} (Filename: {filename})"
            content_parts.append(index_label) 
            
            # 2. Read image bytes
            with open(path, "rb") as f:
                image_bytes = f.read()
            
            # 3. Create a Part object with the correct MIME type
            # Note: Ensure the MIME type matches your actual files (image/jpeg or image/png)
            image_part = types.Part.from_bytes(
                data=image_bytes,
                mime_type="image/jpeg" 
            )
            content_parts.append(image_part)
            
        except Exception as e:
            print(f"Error reading file {path}: {e}")
            return None

    # B. Main Text Prompt (Placed AFTER all images)
    # This instructs the model to synthesize information from the indexed images above.
    prompt_text = """
    You are an expert data entry assistant for Thai Election documents (Form S.S. 5/18).
    
    Instructions:
    1. Analyze the sequence of images labeled Page 1, Page 2, etc. provided above. These pages belong to the SAME single report.
    2. Extract information strictly according to the JSON schema provided.
    3. Consolidate data from all pages. 
       - The header information (District, Date) is usually on Page 1.
       - The 'Vote Results' table often spans across multiple pages. Merge them into a single list.
    4. Validation: Ensure the 'total ballots used' matches the sum of 'good', 'bad', and 'no vote' ballots.
    5. Form Type: Detect if this is a 'Constituency' form (candidates with names) or 'PartyList' form (party names only).
    """
    content_parts.append(prompt_text)

    # C. Send Request to Gemini
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash", 
            contents=content_parts,   # List structure: [Index, Img, Index, Img, ..., Main Prompt]
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=election_form_schema,
                temperature=0.0, # Low temperature for factual extraction
            ),
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"Error calling Gemini: {e}")
        return None

# --- 3. Schema Definition ---
election_form_schema = {
    "type": "OBJECT",
    "properties": {
        "form_info": {
            "type": "OBJECT",
            "description": "Header information identifying the polling station.",
            "properties": {
                "date": {"type": "STRING", "description": "Date of election (e.g., 14 May 2566)"},
                "province": {"type": "STRING", "description": "Province name"},
                "district": {"type": "STRING", "description": "District name (Amphoe/Khet)"},
                "constituency_number": {"type": "STRING", "description": "Constituency number"},
                "polling_station_number": {"type": "STRING", "description": "Unit/Polling Station number"},
                "form_type": {
                    "type": "STRING",
                    "enum": ["Constituency", "PartyList"],
                    "description": "Type of form: Constituency (Candidates) or PartyList (Parties)"
                }
            },
            "required": ["province", "district", "polling_station_number"]
        },
        "ballot_statistics": {
            "type": "OBJECT",
            "description": "Section 2: Ballot accounting statistics.",
            "properties": {
                "ballots_used": {"type": "INTEGER", "description": "Item 2.2: Total used ballots"},
                "good_ballots": {"type": "INTEGER", "description": "Item 2.2.1: Valid ballots"},
                "bad_ballots": {"type": "INTEGER", "description": "Item 2.2.2: Void/Spoiled ballots"},
                "no_vote_ballots": {"type": "INTEGER", "description": "Item 2.2.3: No Vote ballots"}
            }
        },
        "vote_results": {
            "type": "ARRAY",
            "description": "Section 3: Vote counts table. Extract all rows from all pages.",
            "items": {
                "type": "OBJECT",
                "properties": {
                    "number": {"type": "INTEGER", "description": "Candidate/Party Number"},
                    "name": {"type": "STRING", "description": "Candidate Name or Party Name"},
                    "vote_count": {"type": "INTEGER", "description": "Vote count (numeric)"},
                    "vote_count_text": {"type": "STRING", "description": "Vote count (Thai written text)"}
                }
            }
        }
    },
    "required": ["form_info", "ballot_statistics", "vote_results"]
}

In [30]:
# --- 4. Main Execution Block ---
# Configuration
# Ensure this path points to your actual image directory
IMAGE_DIR = "assets/ss5-18/ss5-18-images/"
DISTRICT_NAME = "บางพลัด25" # District name to filter by (can be part of the filename)

# Initialize the Gemini Client (Credentials are auto-detected from env vars)
client = genai.Client()

print(f"Scanning directory: {IMAGE_DIR} for district: {DISTRICT_NAME}...")

# Get images grouped by Report ID and sorted by page
report_groups = get_district_image_groups(IMAGE_DIR, DISTRICT_NAME)

if not report_groups:
    print("No images found matching the criteria.")
else:
    # Iterate through each report group (e.g., Report "BangPhlat25" with 6 pages)
    for report_id, pages in report_groups.items():
        print("-" * 50)
        print(f"Processing Report ID: {report_id}")
        print(f"Found {len(pages)} pages: {[os.path.basename(p) for p in pages]}")
        
        # Extract data from the multi-page document
        result = extract_from_multipage_report(pages, client)
        
        if result:
            print("Extraction Success!")
            # Print the result nicely formatted
            print(json.dumps(result, indent=2, ensure_ascii=False))
        else:
            print("Failed to extract data.")

Scanning directory: assets/ss5-18/ss5-18-images/ for district: บางพลัด25...
--------------------------------------------------
Processing Report ID: บางพลัด25
Found 6 pages: ['บางพลัด25_page1.jpg', 'บางพลัด25_page2.jpg', 'บางพลัด25_page3.jpg', 'บางพลัด25_page4.jpg', 'บางพลัด25_page5.jpg', 'บางพลัด25_page6.jpg']
Extraction Success!
{
  "form_info": {
    "date": "14 พฤษภาคม 2566",
    "province": "กรุงเทพมหานคร",
    "district": "บางพลัด",
    "constituency_number": "25",
    "polling_station_number": "25",
    "form_type": "Constituency"
  },
  "ballot_statistics": {
    "ballots_used": 433,
    "good_ballots": 415,
    "bad_ballots": 6,
    "no_vote_ballots": 12
  },
  "vote_results": [
    {
      "number": 1,
      "name": "นายจักรพันธ์ พรหมิภร",
      "vote_count": 17,
      "vote_count_text": "สิบเจ็ด"
    },
    {
      "number": 2,
      "name": "นายกฤษณ์ สุริยผล",
      "vote_count": 2,
      "vote_count_text": "สอง"
    },
    {
      "number": 3,
      "name": "นางทัศวรรณ รุจ

failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces: client error (Connect)
